In [5]:
import numpy as np
import cv2

In [6]:
cap = cv2.VideoCapture('C:/Users/ruqaiya rangwala/Downloads/Major Project/walking.avi')
#cap.set(3, 200);
#cap.set(4, 200);
feature_params = dict(maxCorners = 100,
                     qualityLevel = 0.3,
                     minDistance = 7,
                     blockSize=7)

lucas_kanade_params = dict(winSize = (15,15),
                          maxLevel = 2,
                          criteria = (cv2.TERM_CRITERIA_EPS| cv2.TERM_CRITERIA_COUNT, 10,0.03))

color = np.random.randint(0,255,(100,3))

ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

prev_corners = cv2.goodFeaturesToTrack(prev_gray, mask = None, **feature_params)
mask = np.zeros_like(prev_frame)

while(1):
    
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    new_corners, status, errors = cv2.calcOpticalFlowPyrLK(prev_gray,
                                                           frame_gray,
                                                           prev_corners,
                                                           None,
                                                           **lucas_kanade_params)
    good_new = new_corners[status==1]
    good_old = prev_corners[status==1]
    
    for i, (new,old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = new.ravel()
        mask = cv2.line(mask, (a,b),(c,d),color[i].tolist(),thickness=2)
        frame = cv2.circle(frame, (a,b),5,color[i].tolist(),-1)
        
    img = cv2.add(frame, mask)
    
    cv2.imshow('Optical Flow - Lucas-Kanade', img)
    
    if cv2.waitKey(1)==13:
        break
    prev_gray = frame_gray.copy()
    prev_corners = good_new.reshape(-1,1,2)
cap.release()
cv2.destroyAllWindows()

Dense Optical Flow

In [7]:
import cv2
import numpy as np

In [4]:
cap = cv2.VideoCapture('C:/Users/ruqaiya rangwala/Downloads/walking.avi')

ret, first_frame = cap.read()
previous_gray = cv2.cvtColor(first_frame,cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(first_frame)
hsv[...,1] = 255

while True:
    
    ret,frame2 = cap.read()
    next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    
    flow = cv2.calcOpticalFlowFarneback(previous_gray, next, None, 0.5,3,15,3,5,1.2,0)
    
    magnitude, angle = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = angle * (180/(np.pi/2))
    hsv[...,2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    final = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    
    cv2.imshow("Dense Optical Flow", final)
    if cv2.waitKey(1)==13:
        break
        
    previous_gray = next
    
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.0.0) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:181: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
